In [3]:
import pandas as pd
import nltk
file_path = r"narod_without_huebor.csv"  
chunk_size = 5
#with open(file_path, 'r', encoding = "utf8") as file:
#   while True:
#        chunk = file.read(chunk_size)
#        if not chunk:
#            break
#print(file)

In [144]:
chunksize = 5
TextFileReader = pd.read_csv(file_path, chunksize=chunksize)
datf = pd.concat(TextFileReader, ignore_index=True)

In [145]:
datf.head()

,Unnamed: 0,page_id,site_id,page_link,page_parent,page_title,page_text,topic
0,0,3,3,http://dzhouniods.narod.ru/,3,Шахматы,Шахматы Шахматы РГСУ online ru Шахматы искусст...,Литература
1,1,2419,22,http://kotbruce.narod.ru/,2419,Британский лиловый короткошерстный кот Bruce G...,Британский лиловый короткошерстный кот Bruce G...,Питомцы
2,2,2434,25,http://sinii-mimrik.narod.ru/,2434,Синие мымрики,Синие мымрики Все о Синих Мымриках Происхожден...,Питомцы
3,3,2435,25,http://sinii-mimrik.narod.ru/2.html,2434,Размножение,Размножение Брачный период Синих мымриков На г...,Питомцы
4,4,2436,25,http://sinii-mimrik.narod.ru/3.html,2434,Повадки синих мымриков,Повадки синих мымриков Повадки Синих Мымриков ...,Питомцы


In [146]:
import re
def clean_n(text):
    if isinstance(text, str):
        return re.sub(r'[\n|\r]', ' ', text)
    return text
datf['page_text'] = datf['page_text'].apply(clean_n)
 

In [147]:
 
def clean_ssylk(text):
    if isinstance(text, str):
        return re.sub(r'[www|http].+[ru|com|org]', '', text)
    return text
 

In [148]:
 
def clean_axan(text):
    if isinstance(text, str):
        return re.sub(r'&.{4};', '', text)
    return text


In [149]:
 
def clean_ssylk2(text):
    if isinstance(text, str):
        return re.sub(r'=\d{9}', '', text)
    return text
 

In [150]:
 
def clean_edit(text):
    if isinstance(text, str):
        return re.sub(r'\[править\]', '', text)
    return text
 

In [151]:
 
def clean_softlist(text):
    if isinstance(text, str):
        return re.sub(r'[A-Z]{5}ist|List|SOFT|На главную|Сайт управляется системой.{5}', '', text)
    return text
 

In [152]:

def clean_hash(text):
    if isinstance(text, str):
        return re.sub(r'\.{3}', '', text)
    return text
 

In [153]:
def clean_grenz(text):
    if isinstance(text, str):
        return re.sub(r'([а-я]\.)([А-Я])', r'\1 \2', text)
    return text
 

In [154]:
from nltk.corpus import stopwords #разделить текст по точкам, токенизация, убрать стоп-слова
stops = set(stopwords.words('russian'))
from nltk.tokenize import word_tokenize
def no_stopwords(text):
    tokens=word_tokenize(text)
    filtered=[i for i in tokens if i.lower() not in stops]
    return filtered
datf['page_text']=datf['page_text'].apply(clean_softlist)
datf['page_text']=datf['page_text'].apply(clean_ssylk)
datf['page_text']=datf['page_text'].apply(clean_ssylk2)
datf['page_text']=datf['page_text'].apply(clean_edit)
datf['page_text']=datf['page_text'].apply(clean_hash)
datf['page_text']=datf['page_text'].apply(clean_grenz)
datf['page_text']=datf['page_text'].astype(str)
datf['page_text']=datf['page_text'].apply(no_stopwords)

In [155]:
datf['page_text']=datf['page_text'].apply(lambda x: " ".join(x))

In [156]:
datf['page_text'][0]

'Шахматы Шахматы РГСУ online ru Шахматы искусство Шахматы спорт Шахматы это работа Шахматы отдых Единственный способ поумнеть играть умным противником Шахматы настольная логическая игра сочетающая элементы искусства науки спорта Одна древнейших игр Земле сохранившихся нашего времени долгое время считалась игрой королей аристократов Играется двумя игроками определённым правилам Ежегодно мире проводятся тысячи различных соревнований шахматам Широко распространено мнение шахматы способствуют развитию умственных способностей памяти творческого мышления Христианская церковь против шахмат момента появления шахмат резко негативную позицию отношению заняла христианская церковь Шахматы приравнивались азартным играм пьянству Примечательно едины представители различных направлений христианства 1161 году католический кардинал Дамиани издал указ запрете игры шахматы среди духовенства своём письме папе Александру назвал шахматы измышлением дьявола игрой непристойной неприемлемой Основатель ордена та

## Запускаем лемматизацию

In [157]:
from tqdm import tqdm

In [158]:
from pymystem3 import Mystem 
import os, json
mystem = Mystem()

def lemmy(text):
    lemmed=mystem.lemmatize(text)
    return lemmed


In [159]:
datf.shape

(42069, 8)

In [160]:
for i in tqdm(range(0, len(datf), 20)):
#for i in tqdm(range(0, 5000, 20)):
    if i < len(datf) - 20:
        j = i+20
    else:
        j = len(datf)
    #datf['page_text'][i:j] = datf['page_text'][i:j].apply(lemmy)
    datf['page_text'][i:j].apply("".join).apply(lemmy).to_csv("lemmatizedn1.csv", mode='a', header=False, index=False, encoding="utf-8")

100%|██████████████████████████████████████████████████████████████████████████████| 250/250 [2:23:05<00:00, 34.34s/it]


## Действия на случай если вы запускали лемматизацию несколько раз, и результаты в отдельных файлах, и вы хотите объединить

In [16]:

df1 = pd.read_csv("lemmatizedn1.csv")
df2 = pd.read_csv("lemmatizedn4.csv")
df3 = pd.read_csv("lemmatizedn3.csv")
df4 = pd.read_csv("lemmatizedn2.csv")
#df_big = pd.concat([df1, df2], ignore_index=True)
#df_big.to_csv("lemmatized.csv")

In [17]:
import re
def fakelist_to_str(fakelist):
    x =  re.sub(r"[\[\]',]", "", fakelist)
    return re.sub(r"   ", " ", x)

In [78]:
def append_to_big(big_df, df):
    dflist = []
    for i in df.iterrows():
        dflist.append("".join((i[1])))
    dfcleanlist = [fakelist_to_str(i) for i in dflist]
    dfpart = pd.DataFrame(dfcleanlist)
    return pd.concat([big_df, dfpart], axis=0, ignore_index=True)

In [80]:
big = pd.DataFrame()
big = append_to_big(big, df1)
big = append_to_big(big, df2)
big = append_to_big(big, df3)
big = append_to_big(big, df4)

In [94]:
big.to_csv("good_lemma_all.csv")

In [82]:
#test_df = big.head(1000)
#test_df.to_csv("small_lemma.csv")
df = pd.read_csv("good_lemma_all.csv")

In [81]:
big.head()

,0
0,британский лиловый короткошерстный кот Bruce G...
1,синий мымрик синий мымрик происхождение размно...
2,размножение брачный период синий мымрик повадк...
3,повадка синий мымрик повадка синий мымрик Micr...
4,вид синий мымрик повадка Microsoft MSDN одногл...


In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
vectorizer = TfidfVectorizer(max_features=500)
#vectors = vectorizer.fit_transform(test_df[0], max_features=500)
vectors = vectorizer.fit_transform(big[0])

In [92]:
vectors.shape

(1000, 500)

In [47]:
import pickle

In [103]:
with open("total_vectorized_500.pkl", "wb") as f:
    pickle.dump(vectors, f)

In [104]:
vects_arr = vectorss.toarray()
with open("array_small_vectorized_500.pkl", "wb") as f:
    pickle.dump(vects_arr, f)